In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import numpy as np
from matplotlib.image import imread
from glob import glob
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:
!pip show tensorflow

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [5]:
IMAGE_SIZE = [224, 224]

In [6]:
train_path = 'casia-amin'

In [7]:
# from tensorflow.keras.applications import EfficientNetB0
# eff = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

from tensorflow.keras.applications import ResNet50V2
model = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# model = tf.keras.applications.ResNet50V2(
#         include_top=False,
#         weights=None,
#         input_tensor=None,
#         input_shape=IMAGE_SIZE + [3],
#         pooling=None,
#         classes=1000,
#         classifier_activation="softmax")
         

In [ ]:
# from tensorflow.keras.applications import EfficientNetB1
# eff1 = EfficientNetB1(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# from tensorflow.keras.applications import EfficientNetB2
# eff2 = EfficientNetB2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# from tensorflow.keras.applications import EfficientNetB3
# eff3 = EfficientNetB3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# from tensorflow.keras.applications import EfficientNetB4
# eff34 = EfficientNetB4(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# from tensorflow.keras.applications import EfficientNetB5
# eff5 = EfficientNetB5(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# from tensorflow.keras.applications import VGG19
# vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
np.array(model.layers[-7].output.name)

In [8]:
for layer in model.layers:
    layer.trainable = True

In [9]:
folders = glob('casia-amin/*')

In [10]:
from tensorflow.keras.initializers import glorot_uniform
headModel = model.output
headModel = Flatten()(headModel)
headModel=Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel=Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(headModel)
headModel = Dense( len(folders) ,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(headModel)

In [ ]:
# from tensorflow.keras.layers import Flatten, Dense, Dropout
# x = Flatten()(model.output)
# # x = Dense(1000, activation='relu')(x)
# prediction = Dense(len(folders), activation='softmax')(x)

In [11]:
# model = Model(inputs=model.input, outputs=prediction)

model = Model(inputs=model.input, outputs=headModel)

# view the structure of the model
model.summary()

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
    
model.compile(
  loss='categorical_crossentropy' ,
  optimizer=opt,
  metrics=['accuracy'])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

# test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 subset = 'training')

test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            subset='validation')

In [15]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [16]:
mc = ModelCheckpoint('/best_model.h5', monitor='val_accuracy' )

In [18]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=70,
    callbacks=[mc,es]
      )

In [19]:
from tensorflow.keras.models import load_model

model.save('model_Resnet50v2_70epoch_.58valaccuracy .h5')

In [20]:
def compute_loss(input_image, filter_index):
    activation = feature_extractor(input_image)
    # We avoid border artifacts by only involving non-border pixels in the loss.
    filter_activation = activation[:, 2:-2, 2:-2, filter_index]
    return tf.reduce_mean(filter_activation)


def gradient_ascent_step(img, filter_index):
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img, filter_index)
    # Compute gradients.
    grads = tape.gradient(loss, img)
    return loss, grads

#simple image scaling to (nR x nC) size
def scale(im, nR, nC):
  nR0 = len(im)     # source number of rows 
  nC0 = len(im[0])  # source number of columns 
  return [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]

In [21]:
import cv2
import glob

# def load_images_from_folder(folder):
#     images = []
#     for filename in glob(folder):
#         img = cv2.imread(os.path.join(folder,filename))
#         if img is not None:
#             images.append(img)
#     return images
images=[]
images_dir = glob.glob("selected/*")
for dir_ in images_dir:
    images.append(cv2.imread(dir_))

In [22]:
print(images_dir)

In [ ]:
# from tensorflow.keras.vis import utils
# !pip show keras_vis


In [ ]:
# !pip show keras

In [ ]:
# from tensorflow.keras import utils

In [ ]:
# from tf_keras_vis import utils
# # from tf.keras.vis.utils import utils
# import vis
# import vis.utils as utils


In [23]:
def reverse_enumerate(iterable):
    
    return zip(reversed(range(len(iterable))), reversed(iterable))


In [24]:
def find_layer_idx(model, layer_name):
 
    layer_idx = None
    for idx, layer in enumerate(model.layers):
        if layer.name == layer_name:
            layer_idx = idx
            break
    if layer_idx is None:
        raise ValueError("No layer with name '{}' within the model".format(layer_name))
    return layer_idx

In [25]:
dead_filters = {}

for img in images:
    img_tf = tf.convert_to_tensor(scale(img, 224, 224), np.float32)
    img_tf = tf.reshape(img_tf, [-1, 224, 224, 3])

    for layer in model.layers:

        layer_name = layer.name
        
        feature_extractor = keras.Model(inputs = model.inputs, outputs = layer.output)

        if layer_name[-layer_name[::-1].find('_'):] == 'relu':

            layer_idx =find_layer_idx(model, layer_name) -1
            
            dead = []
            for filter_index in range (layer_idx):
                loss, grad = gradient_ascent_step(img_tf, filter_index)
                
                if tf.reduce_sum(grad) <= 0.02 : #==0
                    print(loss, grad.shape, ' Dead filter detected!',  filter_index) #layer.name,
                    # print(tf.math.count_nonzero(grad))
                    dead.append(filter_index)

#             print('ReLU name: {}, ReLU index: {}'.format(layer_name, layer_idx +1))

            while True:

                conv = model.layers[layer_idx].name

                if conv[-conv[::-1].find('_'):] == 'conv':

                    conv_idx = find_layer_idx(model, conv)
                    
                    

#                     print('Conv name: {}, Conv index: {}'.format(conv, conv_idx))

#                     print('---')

                    break

                layer_idx -= 1
            
            
            if dead != []:
                if f'{conv}' in dead_filters.keys():
                    dead_filters[f'{conv}'].append(dead)
                else:
                    dead_filters[f'{conv}'] = dead
            
# for layer in model.layers:

#     layer_name = layer.name

#     if layer_name[-layer_name[::-1].find('_'):] == 'relu':

#         layer_idx =find_layer_idx(model, layer_name) -1

#         print('ReLU name: {}, ReLU index: {}'.format(layer_name, layer_idx +1))

#         while True:

#             conv = model.layers[layer_idx].name

#             if conv[-conv[::-1].find('_'):] == 'conv':

#                 conv_idx = find_layer_idx(model, conv)

#                 print('Conv name: {}, Conv index: {}'.format(conv, conv_idx))

#                 print('---')

#                 break

#             layer_idx -= 1
            

In [26]:
np.save('dead_filters_resnet50v2_100image_.02grad.npy',dead_filters)

In [27]:
dead_filters


In [ ]:
images_dir

In [ ]:
# dead_filters = {}

# for img in images:
#     img_tf = tf.convert_to_tensor(scale(img, 224, 224), np.float32)
#     img_tf = tf.reshape(img_tf, [-1, 224, 224, 3])

#     for layer in model.layers:
#         print('Layer in Network:', layer.name)
#         feature_extractor = keras.Model(inputs = model.inputs, outputs = layer.output)

#         if layer.get_config().get('activation', None) == 'relu':
#             print('Number of filters in layer:',layer.get_config().get('filters', None))
#             num_filters = layer.get_config().get('filters', None)
            
#             dead = []
#             for filter_index in range (num_filters):
#                 loss, grad = gradient_ascent_step(img_tf, filter_index)
                
#                 if tf.reduce_sum(grad) <= 0.02 : #==0
#                     print(loss, grad.shape, ' Dead filter detected!', layer.name, filter_index)
#                     # print(tf.math.count_nonzero(grad))
#                     dead.append(filter_index)
#             if dead != []:
#                 if f'{layer.name}' in dead_filters.keys():
#                     dead_filters[f'{layer.name}'].append(dead)
#                 else:
#                     dead_filters[f'{layer.name}'] = dead

In [28]:
def most_repeated(dictionary = {}, threshold = 0):
    dic = {}
    for key in dictionary.keys():
        temp = []
        for i in range(len(dictionary[key])):
            if type(dictionary[key][i]) == int:
                dictionary[key][i] = [dictionary[key][i]]
            temp.extend(dictionary[key][i])
        n, m = np.unique(temp, return_counts = True)
        dic[key] = n, m
    over_th = {}
    for key in dic:
        for i in range(len(dic[key][0])):
            if dic[key][1][i] >= threshold:
                if key in over_th.keys():
                    over_th[key] = np.append(over_th[key], dic[key][0][i])
                else:
                    over_th.update({key: dic[key][0][i]})
        if key in over_th.keys():
            if type(over_th[key])== np.int32:
                over_th[key] = np.array([over_th[key]])
    return over_th

In [29]:
dead_nerouns_new = most_repeated(dead_filters,70)

In [30]:
np.save('dead_neroun_new_resnet50v2_100image_.02grad.npy',dead_nerouns_new)

In [31]:
dead_nerouns_new

In [32]:
import kerassurgeon
from kerassurgeon.operations import delete_channels

In [ ]:
# # dead_filters_name = [layer for layer in dead_nerouns_new]
# # dead_filters_idx = [*dead_nerouns_new.values()]
# # model.get_layer(model.get_layer(dead_filters_name[0]), dead_filters_idx[0]), dead_filters_idx[0]
# print(model.get_layer(dead_filters_name[1]))
# # new_model = delete_channels(model, model.get_layer(dead_filters_name[0]), dead_filters_idx[0])

In [33]:
dead_filters_name = [layer for layer in dead_nerouns_new]
dead_filters_idx = [*dead_nerouns_new.values()]
# new_model = delete_channels(model, model.get_layer(dead_filters_name[0]), dead_filters_idx[0])
# for i in range(len(dead_filters_idx)-1):
#     new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[i+1]), dead_filters_idx[i+1])
# new_model.summary()


# new_model = delete_channels(model,model.get_layer('conv1_conv'), [1, 2, 3, 4])



In [35]:
new_model = delete_channels(model, model.get_layer(dead_filters_name[0]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[1]), dead_filters_idx[1])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[2]), dead_filters_idx[2])
# new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[3]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[4]), dead_filters_idx[4])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[5]), dead_filters_idx[5])
# new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[6]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[7]), dead_filters_idx[7])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[8]), dead_filters_idx[8])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[9]), dead_filters_idx[0])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[10]), dead_filters_idx[10])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[11]), dead_filters_idx[11])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[12]), dead_filters_idx[0])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[13]), dead_filters_idx[13])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[14]), dead_filters_idx[14])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[15]), dead_filters_idx[0])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[16]), dead_filters_idx[16])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[17]), dead_filters_idx[17])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[18]), dead_filters_idx[0])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[19]), dead_filters_idx[19])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[20]), dead_filters_idx[20])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[21]), dead_filters_idx[0])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[22]), dead_filters_idx[22])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[23]), dead_filters_idx[23])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[24]), dead_filters_idx[24])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[25]), dead_filters_idx[25])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[25]), dead_filters_idx[26])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[27]), dead_filters_idx[27])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[28]), dead_filters_idx[28])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[29]), dead_filters_idx[29])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[30]), dead_filters_idx[30])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[31]), dead_filters_idx[31])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[32]), dead_filters_idx[32])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[33]), dead_filters_idx[33])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[34]), dead_filters_idx[34])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[35]), dead_filters_idx[35])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[36]), dead_filters_idx[36])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[37]), dead_filters_idx[37])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[38]), dead_filters_idx[38])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[39]), dead_filters_idx[39])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[40]), dead_filters_idx[40])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[41]), dead_filters_idx[41])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[42]), dead_filters_idx[42])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[43]), dead_filters_idx[43])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[44]), dead_filters_idx[44])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[45]), dead_filters_idx[45])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[46]), dead_filters_idx[46])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[47]), dead_filters_idx[47])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[48]), dead_filters_idx[48])


In [36]:
# new_model = delete_channels(model, model.get_layer(dead_filters_name[9]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[10]), dead_filters_idx[10])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[11]), dead_filters_idx[11])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[12]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[13]), dead_filters_idx[13])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[14]), dead_filters_idx[14])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[15]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[16]), dead_filters_idx[16])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[17]), dead_filters_idx[17])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[18]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[19]), dead_filters_idx[19])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[20]), dead_filters_idx[20])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[21]), dead_filters_idx[0])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[22]), dead_filters_idx[22])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[23]), dead_filters_idx[23])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[24]), dead_filters_idx[24])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[25]), dead_filters_idx[25])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[26]), dead_filters_idx[26])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[27]), dead_filters_idx[27])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[28]), dead_filters_idx[28])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[29]), dead_filters_idx[29])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[30]), dead_filters_idx[30])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[31]), dead_filters_idx[31])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[32]), dead_filters_idx[32])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[33]), dead_filters_idx[33])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[34]), dead_filters_idx[34])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[35]), dead_filters_idx[35])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[36]), dead_filters_idx[36])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[37]), dead_filters_idx[37])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[38]), dead_filters_idx[38])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[39]), dead_filters_idx[39])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[40]), dead_filters_idx[40])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[41]), dead_filters_idx[41])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[42]), dead_filters_idx[42])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[43]), dead_filters_idx[43])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[44]), dead_filters_idx[44])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[45]), dead_filters_idx[45])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[46]), dead_filters_idx[46])
new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[47]), dead_filters_idx[47])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[48]), dead_filters_idx[48])


In [37]:
new_model.summary()

In [38]:
opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
    
new_model.compile(
  loss='categorical_crossentropy' ,
  optimizer=opt,
  metrics=['accuracy'])

In [39]:
mcs = ModelCheckpoint('/best_new_model.h5', monitor='val_accuracy' )

In [41]:
z = new_model.fit(
    training_set,
    validation_data=test_set,
    epochs=30,
     callbacks=[mcs,es]   )





In [43]:
dead_filters_new_model = {}

for img in images:
    img_tf = tf.convert_to_tensor(scale(img, 224, 224), np.float32)
    img_tf = tf.reshape(img_tf, [-1, 224, 224, 3])

    for layer in new_model.layers:

        layer_name = layer.name
        
        feature_extractor = keras.Model(inputs = new_model.inputs, outputs = layer.output)

        if layer_name[-layer_name[::-1].find('_'):] == 'relu':

            layer_idx =find_layer_idx(new_model, layer_name) -1
            
            dead = []
            for filter_index in range (layer_idx):
                loss, grad = gradient_ascent_step(img_tf, filter_index)
                
                if tf.reduce_sum(grad) <= 0.02 : #==0
                    print(loss, grad.shape, ' Dead filter detected!',  filter_index) #layer.name,
                    # print(tf.math.count_nonzero(grad))
                    dead.append(filter_index)

#             print('ReLU name: {}, ReLU index: {}'.format(layer_name, layer_idx +1))

            while True:

                conv = new_model.layers[layer_idx].name

                if conv[-conv[::-1].find('_'):] == 'conv':

                    conv_idx = find_layer_idx(new_model, conv)
                    
                    

#                     print('Conv name: {}, Conv index: {}'.format(conv, conv_idx))

#                     print('---')

                    break

                layer_idx -= 1
            
            
            if dead != []:
                if f'{conv}' in dead_filters_new_model.keys():
                    dead_filters_new_model[f'{conv}'].append(dead)
                else:
                    dead_filters_new_model[f'{conv}'] = dead
            
# for layer in model.layers:

#     layer_name = layer.name

#     if layer_name[-layer_name[::-1].find('_'):] == 'relu':

#         layer_idx =find_layer_idx(model, layer_name) -1

#         print('ReLU name: {}, ReLU index: {}'.format(layer_name, layer_idx +1))

#         while True:

#             conv = model.layers[layer_idx].name

#             if conv[-conv[::-1].find('_'):] == 'conv':

#                 conv_idx = find_layer_idx(model, conv)

#                 print('Conv name: {}, Conv index: {}'.format(conv, conv_idx))

#                 print('---')

#                 break

#             layer_idx -= 1
            

In [44]:
np.save('dead_filters_new_model_resnet50v2_100image_.02grad.npy',dead_filters_new_model)

In [45]:
dead_filters_new_model

In [46]:
dead_nerouns_new_model = most_repeated(dead_filters_new_model,80)

In [47]:
np.save('dead_neroun_new_model_resnet50v2_100image_.02grad.npy',dead_nerouns_new_model)

In [48]:
dead_nerouns_new_model

In [49]:
dead_filters_name = [layer for layer in dead_nerouns_new_model]
dead_filters_idx = [*dead_nerouns_new_model.values()]

In [50]:
dead_filters_name

In [51]:
new_model_1 = delete_channels(new_model, new_model.get_layer(dead_filters_name[0]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[1]), dead_filters_idx[1])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[2]), dead_filters_idx[2])
# new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[3]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[4]), dead_filters_idx[4])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[5]), dead_filters_idx[5])
# new_model = delete_channels(new_model, new_model.get_layer(dead_filters_name[6]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[7]), dead_filters_idx[7])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[8]), dead_filters_idx[8])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[9]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[10]), dead_filters_idx[10])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[11]), dead_filters_idx[11])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[12]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[13]), dead_filters_idx[13])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[14]), dead_filters_idx[14])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[15]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[16]), dead_filters_idx[16])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[17]), dead_filters_idx[17])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[18]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[19]), dead_filters_idx[19])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[20]), dead_filters_idx[20])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[21]), dead_filters_idx[0])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[22]), dead_filters_idx[22])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[23]), dead_filters_idx[23])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[24]), dead_filters_idx[24])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[25]), dead_filters_idx[25])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[26]), dead_filters_idx[26])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[27]), dead_filters_idx[27])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[28]), dead_filters_idx[28])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[29]), dead_filters_idx[29])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[30]), dead_filters_idx[30])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[31]), dead_filters_idx[31])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[32]), dead_filters_idx[32])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[33]), dead_filters_idx[33])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[34]), dead_filters_idx[34])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[35]), dead_filters_idx[35])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[36]), dead_filters_idx[36])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[37]), dead_filters_idx[37])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[38]), dead_filters_idx[38])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[39]), dead_filters_idx[39])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[40]), dead_filters_idx[40])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[41]), dead_filters_idx[41])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[42]), dead_filters_idx[42])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[43]), dead_filters_idx[43])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[44]), dead_filters_idx[44])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[45]), dead_filters_idx[45])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[46]), dead_filters_idx[46])
new_model_1 = delete_channels(new_model_1, new_model_1.get_layer(dead_filters_name[47]), dead_filters_idx[47])
# new_model = delete_channels(model, model.get_layer(dead_filters_name[48]), dead_filters_idx[48])


In [52]:
new_model_1.summary()

In [53]:
opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
    
new_model_1.compile(
  loss='categorical_crossentropy' ,
  optimizer=opt,
  metrics=['accuracy'])

In [54]:
mcss = ModelCheckpoint('/best_new_model_1.h5', monitor='val_accuracy' )

In [55]:
z = new_model_1.fit(
    training_set,
    validation_data=test_set,
    epochs=60,
     callbacks=[mcss,es]   )





In [57]:
z = new_model_1.fit(
    training_set,
    validation_data=test_set,
    epochs=40)
     #callbacks=[mcss,es]   )

In [ ]:
for layer in model.layers:

    layer_name = layer.name

    if layer_name[-layer_name[::-1].find('_'):] == 'relu':

        layer_idx =find_layer_idx(model, layer_name) -1

        print('ReLU name: {}, ReLU index: {}'.format(layer_name, layer_idx +1))

        while True:

            conv = model.layers[layer_idx].name

            if conv[-conv[::-1].find('_'):] == 'conv':

                conv_idx = find_layer_idx(model, conv)

                print('Conv name: {}, Conv index: {}'.format(conv, conv_idx))

                print('---')

                break

            layer_idx -= 1